# 0. Imports and Setting up Anthropic API Client

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-dotenv

import os
import dotenv

dotenv.load_dotenv('/content/drive/MyDrive/.env')

True

In [3]:
# Load Prompts and Problem Description
# Variables Prompt
prompt11_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt11_MathematicalModel.txt'

# Objective Prompt
prompt12_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt12_MathematicalModel.txt'

# Constraint Prompt
prompt13_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt13_MathematicalModel.txt'

# Code Prompt
prompt2_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt2_PyomoCode.txt'
problem_desc_path = '/content/drive/MyDrive/Thesis/ProblemDescriptions/MIP/MIP4.txt'

prompt11_file = open(prompt11_path, "r")
prompt12_file = open(prompt12_path, "r")
prompt13_file = open(prompt13_path, "r")
prompt2_file = open(prompt2_path, "r")
problem_desc_file = open(problem_desc_path, "r")

prompt11 = prompt11_file.read()
print("Prompt 1.1 (Variables):\n", prompt11)

prompt12 = prompt12_file.read()
print("Prompt 1.2 (Objctive):\n", prompt12)

prompt13 = prompt13_file.read()
print("Prompt 1.3 (Constraints):\n", prompt13)

prompt2 = prompt2_file.read()
print("Prompt 2:\n", prompt2)

problem_desc = problem_desc_file.read()
print("Problem Description:\n", problem_desc)

Prompt 1.1 (Variables):
 Please formulate only the variables for this mathematical optimization problem. 
Prompt 1.2 (Objctive):
 Please formulate only the objective function for this mathematical optimization problem. 
Prompt 1.3 (Constraints):
 Please formulate only the constraints for this mathematical optimization problem. 
Prompt 2:
 Please write a python pyomo code for this optimization problem.
Use sample data where needed.
Indicate where you use sample data.
Problem Description:
 You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in cas

In [4]:
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.7/862.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00


In [5]:
# Importing Anthropic & Setting Headers
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

# 1. Prompt 1.1: Create Variables for Mathematical Model

In [6]:
message11 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt11,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": problem_desc}
    ]
)

response_p11 = message11.content[0].text

In [7]:
# Print response
print(response_p11)

Here are the variables for the mathematical optimization problem:

1. Decision variables:
   - x_ij: binary variable indicating if pipe ij is activated (1) or deactivated (0), for ij in {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG}
   - f_ij(t): continuous variable representing the flow in pipe ij at time step t, for ij in {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG} and t in {1, 2, ..., 10}
   - r_i(t): continuous variable representing the amount of water released from point i at time step t, for i in {A, B, C, D, E, F, G} and t in {1, 2, ..., 10}

2. Input parameters:
   - c_ij: capacity of pipe ij, for ij in {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG}
   - f(t): inflow function at point A at time step t, where f(t) = max(-4/30t^3+t^2-0.234t+3, 0) and t in {1, 2, ..., 10}


# 1. Prompt 1.2: Create Objective for Mathematical Model

In [8]:
message12 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt12,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": (problem_desc + response_p11)}
    ]
)

response_p12 = message12.content[0].text

In [9]:
# Print response
print(response_p12)

Here is the formulation of the objective function for the given mathematical optimization problem:

Maximize: Sum(f_EG(t) + f_FG(t) + r_G(t), for t = 1 to 10)

The objective is to maximize the total amount of water that reaches point G over the 10 time steps. This can be achieved by summing up the flow in pipes EG and FG, as well as any water released directly from point G, for each time step from 1 to 10.

In mathematical notation, the objective function can be written as:

Maximize: ∑(t = 1 to 10) [f_EG(t) + f_FG(t) + r_G(t)]

This objective function is subject to the constraints mentioned in the problem description, such as flow conservation at each point, pipe capacity limits, the number of activated pipes, and the inflow function at point A.


# 1. Prompt 1.3: Create Constraints for Mathematical Model


In [10]:
message13 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt13,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": (problem_desc + response_p11 + response_p12)}
    ]
)

response_p13 = message13.content[0].text

In [11]:
# Print response
print(response_p13)

Here are the constraints for the given mathematical optimization problem:

1. Flow conservation constraints:
   For each point i in {A, B, C, D, E, F, G} and time step t in {1, 2, ..., 10}:
   - Point A: f(t) = f_AB(t) + f_AC(t) + f_AF(t) + r_A(t)
   - Point B: f_AB(t-1) + f_FB(t-1) = f_BC(t) + f_BD(t) + r_B(t)
   - Point C: f_AC(t-1) + f_BC(t-1) = f_CE(t) + f_CF(t) + r_C(t)
   - Point D: f_BD(t-1) = f_DE(t) + r_D(t)
   - Point E: f_BE(t-1) + f_CE(t-1) + f_DE(t-1) = f_EG(t) + r_E(t)
   - Point F: f_AF(t-1) + f_CF(t-1) = f_FB(t) + f_FG(t) + r_F(t)
   - Point G: f_EG(t-1) + f_FG(t-1) = r_G(t)

2. Pipe capacity constraints:
   For each pipe ij in {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG} and time step t in {1, 2, ..., 10}:
   - f_ij(t) ≤ c_ij * x_ij

3. Pipe activation constraint:
   Sum(x_ij, for ij in {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG}) = 10

4. Non-negativity constraints:
   - f_ij(t) ≥ 0, for ij in {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG} and t in {1, 2, .

# 2. Prompt 2: Write Code based on Mathematical Model

In case more than the component of each step was generated, use "last instance"/ "newest version" of the components.

Example:
Prompt 1.1 (Variables): Model generates only variables
Prompt 1.2 (Objectvie): Model generates objective and constraints
Prompt 1.3 (Constraints): Model generates only constraints

Use Variables from step 1.1, use objective from 1.2 and use constraints from 1.3 (since constraints version 1.3 is more recent than constraints version 1.2).

This set up allows for potential self-correction of the model throughout the three prompts.

Correctness of each component is still generated in the respective step, so self-correction is independent of this.

In [12]:
message2 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt2,
    max_tokens=4069,
    messages=[
        {"role": "user", "content": (response_p11 + response_p12 + response_p13)}
    ]
)

response_p2 = message2.content[0].text

In [13]:
# Print response
print(response_p2)

Here's a Python Pyomo code for the given optimization problem, using sample data where needed:

```python
from pyomo.environ import *

# Create a Pyomo model
model = ConcreteModel()

# Set of points
model.points = Set(initialize=['A', 'B', 'C', 'D', 'E', 'F', 'G'])

# Set of pipes
model.pipes = Set(initialize=['AB', 'AC', 'AF', 'BC', 'BD', 'BE', 'CE', 'CF', 'DE', 'EG', 'FB', 'FG'])

# Set of time steps
model.time_steps = RangeSet(1, 10)

# Decision variables
model.x = Var(model.pipes, within=Binary)
model.f = Var(model.pipes, model.time_steps, within=NonNegativeReals)
model.r = Var(model.points, model.time_steps, within=NonNegativeReals)

# Input parameters
# Sample data for pipe capacities (replace with actual data if available)
pipe_capacities = {
    'AB': 10, 'AC': 8, 'AF': 12, 'BC': 6, 'BD': 9, 'BE': 7,
    'CE': 5, 'CF': 11, 'DE': 8, 'EG': 10, 'FB': 9, 'FG': 11
}
model.c = Param(model.pipes, initialize=pipe_capacities)

# Inflow function at point A
def inflow(model, t):
    retur

# 4. Input Problem Data and Test Model Code

In [14]:
# Installing pyomo nd solver
!pip install -q pyomo
!pip install pandas
!apt-get install -y -qq glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.3 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121913 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [15]:
from pyomo.environ import *

# Create a Pyomo model
model = ConcreteModel()

# Set of points
model.points = Set(initialize=['A', 'B', 'C', 'D', 'E', 'F', 'G'])

# Set of pipes
model.pipes = Set(initialize=['AB', 'AC', 'AF', 'BC', 'BD', 'BE', 'CE', 'CF', 'DE', 'EG', 'FB', 'FG'])

# Set of time steps
model.time_steps = RangeSet(1, 10)

# Decision variables
model.x = Var(model.pipes, within=Binary)
model.f = Var(model.pipes, model.time_steps, within=NonNegativeReals)
model.r = Var(model.points, model.time_steps, within=NonNegativeReals)

# Input parameters
# Sample data for pipe capacities (replace with actual data if available)
pipe_capacities = {
    'AB': 3, 'AC': 6, 'AF': 1, 'BC': 3, 'BD': 10, 'BE': 4,
    'CE': 4, 'CF': 4, 'DE': 5, 'EG': 5, 'FB': 12, 'FG': 7
}
model.c = Param(model.pipes, initialize=pipe_capacities)

# Inflow function at point A
def inflow(model, t):
    return max(-4/30 * t**3 + t**2 - 0.234 * t + 3, 0)

# Objective function
def objective_rule(model):
    return sum(model.f['EG', t] + model.f['FG', t] + model.r['G', t] for t in model.time_steps)
model.objective = Objective(rule=objective_rule, sense=maximize)

# Constraints
def flow_conservation_rule(model, i, t):
    if i == 'A':
        return inflow(model, t) == model.f['AB', t] + model.f['AC', t] + model.f['AF', t] + model.r['A', t]
    elif i == 'B':
        return model.f['AB', t-1] + model.f['FB', t-1] == model.f['BC', t] + model.f['BD', t] + model.r['B', t]
    elif i == 'C':
        return model.f['AC', t-1] + model.f['BC', t-1] == model.f['CE', t] + model.f['CF', t] + model.r['C', t]
    elif i == 'D':
        return model.f['BD', t-1] == model.f['DE', t] + model.r['D', t]
    elif i == 'E':
        return model.f['BE', t-1] + model.f['CE', t-1] + model.f['DE', t-1] == model.f['EG', t] + model.r['E', t]
    elif i == 'F':
        return model.f['AF', t-1] + model.f['CF', t-1] == model.f['FB', t] + model.f['FG', t] + model.r['F', t]
    elif i == 'G':
        return model.f['EG', t-1] + model.f['FG', t-1] == model.r['G', t]
model.flow_conservation = Constraint(model.points, model.time_steps, rule=flow_conservation_rule)

def pipe_capacity_rule(model, ij, t):
    return model.f[ij, t] <= model.c[ij] * model.x[ij]
model.pipe_capacity = Constraint(model.pipes, model.time_steps, rule=pipe_capacity_rule)

def pipe_activation_rule(model):
    return sum(model.x[ij] for ij in model.pipes) == 10
model.pipe_activation = Constraint(rule=pipe_activation_rule)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print(f"Objective value: {model.objective()}")
print("Activated pipes:")
for ij in model.pipes:
    if model.x[ij].value == 1:
        print(ij)

ERROR:pyomo.core:Rule failed when generating expression for Constraint flow_conservation with index ('B', 1):
KeyError: "Index '('AB', 0)' is not valid for indexed component 'f'"
ERROR:pyomo.core:Constructing component 'flow_conservation' from data=None failed:
    KeyError: "Index '('AB', 0)' is not valid for indexed component 'f'"


KeyError: "Index '('AB', 0)' is not valid for indexed component 'f'"

# 5. Correct The Model Code to Test Mathematical Model (if applicable)